# Run Experiments

In [1]:
import pandas as pd
import json 
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter
import seaborn as sns
import numpy as np
from ema_workbench import TimeSeriesOutcome, perform_experiments, RealParameter, CategoricalParameter, ScalarOutcome, ema_logging, save_results
from ema_workbench.analysis import lines, prim
from ema_workbench.analysis.plotting_util import Density
from ema_workbench.connectors.vensim import VensimModel

C:\Users\marno\miniconda3\Lib\site-packages\ema_workbench\connectors\__init__.py:29: ImportWarning: netlogo connector not available
  warnings.warn("netlogo connector not available", ImportWarning)
C:\Users\marno\miniconda3\Lib\site-packages\ema_workbench\connectors\__init__.py:34: ImportWarning: simio connector not available
  warnings.warn("simio connector not available", ImportWarning)


In [2]:
wd = r'../2_ModelAndParameters'

### Set uncertainties
#### CategoricalParameter uncertainties

In [3]:
CATEGORIES = (0,1)

# Variables that vary across two discrete categories using a switch, lookups specified in the model
categorical_names = ['EBIKE UPTAKE SWITCH', 'TIPPING POINT SWITCH'] 
categoricals = {var: dict(name=var, categories=CATEGORIES) for var in categorical_names} # Dict with input params for CategoricalParameter
categoricals

{'EBIKE UPTAKE SWITCH': {'name': 'EBIKE UPTAKE SWITCH', 'categories': (0, 1)},
 'TIPPING POINT SWITCH': {'name': 'TIPPING POINT SWITCH',
  'categories': (0, 1)}}

#### RealParameter uncertainties

In [4]:
# Uncertainties specified per individual variable
indiv_params = pd.read_csv(wd+'/single_params.csv', sep=';', index_col='variable').to_dict(orient='index')
indiv_dicts = {key: dict(name=key, **value) for key, value in indiv_params.items()} 
indiv_dicts.keys()

dict_keys(['DESIRED PEDESTRIAN SPACE SHARE', 'DESIRED SHARE OF CAR SPACE USED FOR PARKING', 'PARKING SPOTS PER HOME CONSTRUCTED', 'CHANGE IN BIKE PARKING PER ADDED SPACE FOR CYCLISTS', 'REFERENCE SHIFT IN CAR ADOPTION', 'CONTACT RATE', 'BIKESHARING TO PRIVATE BIKE UPGRADE FRACTION', 'PERCEPTION TIME', 'NET ADDED PT CAPACITY PER YEAR', 'DESTINATION SHIFT RATE RING', 'DESTINATION SHIFT RATE CITY'])

In [5]:
# Effect sensitivities have common bounds
BOUNDS = {'lower_bound':0.5, 'upper_bound':2}
sensitivity_names = ['RENEWAL RATE SENSITIVITY', 'CAR ADOPTION SENSITIVITY TO USAGE', 'BIKE ADOPTION SENSITIVITY TO PARKING', 'BIKE ADOPTION SENSITIVITY TO USAGE', 'CAR NETWORK DISTANCE SENSITIVITY', 'CYCLING NETWORK DISTANCE SENSITIVITY', 'SUBJECTIVE CYCLING TIME SENSITIVITY', 'PARKING SEARCH AND EGRESS SENSITIVITY', 'SENSITIVITY TO SIDEWALK DATA UNCERTAINTY', 'SENSITIVITY TO CAR LANE DATA UNCERTAINTY']
sens_dicts = {key: dict(name=key, **BOUNDS) for key in sensitivity_names}

In [6]:
# ema distinguishes real parameters from catagorical parameters
real_dicts = sens_dicts
real_dicts.update(indiv_dicts)

In [7]:
# add = {'CHANGE IN BIKE PARKING PER ADDED SPACE FOR CYCLISTS': {'name': 'CHANGE IN BIKE PARKING PER ADDED SPACE FOR CYCLISTS',
#                                                               'lower_bound': 0,
#                                                               'upper_bound': 2}}
# real_dicts.update(add)
# change = {'CONTACT RATE': {'name': 'CONTACT RATE',
#                                                               'lower_bound': 1,
#                                                               'upper_bound': 3}}
# real_dicts.update(change)
# # real_dicts

In [8]:
cat_params = [CategoricalParameter(**categorical) for categorical in categoricals.values()]
real_params = [RealParameter(**real) for real in real_dicts.values()]

### Load model
Upon publishing the model as .vpmx file it can be read by the ema-workbench connector. Note the ema-workbench connector only works with the DSS version of Vensim.

In [9]:
ema_logging.log_to_stderr(ema_logging.INFO)
model = VensimModel("roadspaceModel", wd=wd, model_file="model.vpmx")

### Set up experiments
Define uncertainties

In [10]:
model.uncertainties = cat_params + real_params

Defining main outcomes to review

In [11]:
main_outcomes = [TimeSeriesOutcome("Car Distance Share", variable_name="car distance share"),
                 TimeSeriesOutcome("Public Transport Distance Share", variable_name="PT distance share"),
                 TimeSeriesOutcome("Bike Distance Share", variable_name="bike distance share"),
                 TimeSeriesOutcome("Walking Distance Share", variable_name="walking distance share")]

In [12]:
other_outcomes = [TimeSeriesOutcome("Variation In Pedestrian Space", variable_name="variation in pedestrian space"),
                  TimeSeriesOutcome("Bike Network Completion", variable_name="bike network completion"),
                  TimeSeriesOutcome("Variation in Subjective Travel Time by Bike", variable_name="variation in subjective travel time by bike"),
                  TimeSeriesOutcome("Variation in Subjective Travel Time by Car", variable_name="variation in subjective travel time by car"),
                  TimeSeriesOutcome("Variation in Subjective Travel Time by PT", variable_name="variation in subjective travel time by PT"),
                  TimeSeriesOutcome("Fraction of City Population with Bike Access", variable_name="fraction of city population with bike access"),
                  TimeSeriesOutcome("Fraction of Ring Population with Bike Access", variable_name="fraction of ring population with bike access"),
                  TimeSeriesOutcome("Population Fraction with Car Access on Long Distance Trips", variable_name="population fraction with car access on long distance trips"),
                  TimeSeriesOutcome("Car Trips Share", variable_name="car trips share"),
                  TimeSeriesOutcome("PT Trips Share", variable_name="PT trips share"),
                  TimeSeriesOutcome("Bike Trips Share", variable_name="bike trips share")]

In [13]:
[outc.name for outc in other_outcomes]

['Variation In Pedestrian Space',
 'Bike Network Completion',
 'Variation in Subjective Travel Time by Bike',
 'Variation in Subjective Travel Time by Car',
 'Variation in Subjective Travel Time by PT',
 'Fraction of City Population with Bike Access',
 'Fraction of Ring Population with Bike Access',
 'Population Fraction with Car Access on Long Distance Trips',
 'Car Trips Share',
 'PT Trips Share',
 'Bike Trips Share']

In [14]:
model.outcomes = main_outcomes + other_outcomes

### Run experiments

In [15]:
experiments, outcomes = perform_experiments(model, 1000)

[MainProcess/INFO] performing 1000 scenarios * 1 policies * 1 model(s) = 1000 experiments
100%|██████████████████████████████████████| 1000/1000 [00:15<00:00, 66.13it/s]
[MainProcess/INFO] experiments finished


#### Variation
Describe the range of variation of the reference modes between initial time and final time

In [16]:
for ref_mode in ["Car Distance Share", "Public Transport Distance Share", "Bike Distance Share"]:
    initial = np.array([ts[0] for ts in outcomes[ref_mode]])
    final = np.array([ts[-1] for ts in outcomes[ref_mode]])
    diff = abs(np.subtract(final, initial))
    max_diff = diff.max() / initial[0]
    min_diff = diff.min() / initial[0]
    print(f'{ref_mode}:\n{min_diff:.0%} - {max_diff:.0%}')

Car Distance Share:
19% - 58%
Public Transport Distance Share:
0% - 20%
Bike Distance Share:
110% - 495%


#### Save results
Save results in a file that can be used for PRIM in a separate notebook

In [17]:
save_results((experiments, outcomes), "./results/results_final.tar.gz")

C:\Users\marno\miniconda3\Lib\site-packages\ema_workbench\em_framework\outcomes.py:541: UserWarning: still to be tested!!
  warnings.warn("still to be tested!!")
[MainProcess/INFO] results saved successfully to C:\Users\marno\Python Projects\paris-roadspace\3_Experiments\results\results_final.tar.gz
